# MAP55672: Case Studies in HPC
## Case 1: Communication-Avoiding QR — Question 1
### Mingjiu Yang
### TSQR Factorisation (4-processor simulation in Python)

This notebook implements the Tall-Skinny QR (TSQR) factorisation of a tall, narrow matrix $A \in \mathbb{R}^{m \times n}$, $m \gg n$, distributed across **4 processors** (simulated via row sub-indexing).

---

### Algorithm Overview

**Level 0 — Local QR on each block:**
$$A_i = Q_i^{(0)} R_i^{(0)}, \quad i = 0, 1, 2, 3$$

**Level 1 — Pairwise reduction (stack and QR):**
$$\begin{bmatrix} R_0^{(0)} \\ R_1^{(0)} \end{bmatrix} = Q_{01}^{(1)} R_{01}^{(1)}, \qquad \begin{bmatrix} R_2^{(0)} \\ R_3^{(0)} \end{bmatrix} = Q_{23}^{(1)} R_{23}^{(1)}$$

**Level 2 — Final reduction:**
$$\begin{bmatrix} R_{01}^{(1)} \\ R_{23}^{(1)} \end{bmatrix} = Q^{(2)} R$$

The $R$ produced at Level 2 is the upper-triangular factor of the full QR decomposition of $A$.

In [ ]:
import numpy as np
from numpy.linalg import qr, norm

np.random.seed(42)  # reproducibility

---
## Part 1: TSQR Implementation

In [ ]:
def tsqr(A, num_procs=4, verbose=True):
    """
    Communication-Avoiding TSQR factorisation.
    Simulates distribution onto `num_procs` processors using row sub-indexing.

    Parameters
    ----------
    A         : ndarray, shape (m, n), m >= n, m divisible by num_procs
    num_procs : int, number of simulated processors (must be power of 2)
    verbose   : bool, print intermediate shapes and residuals

    Returns
    -------
    R_final : upper-triangular factor, shape (n, n)
    Qs      : dict of all Q factors at each level (for full Q reconstruction)
    """
    m, n = A.shape
    assert m % num_procs == 0, "m must be divisible by num_procs"
    assert m >= n, "Matrix must be tall (m >= n)"

    block_size = m // num_procs  # rows per processor

    # ----------------------------------------------------------------
    # Level 0: Local QR on each processor block
    # Each processor i owns rows [ i*block_size : (i+1)*block_size ]
    # ----------------------------------------------------------------
    if verbose:
        print("=" * 55)
        print(f"Input matrix A: {m} x {n}")
        print(f"Block size per processor: {block_size} x {n}")
        print("=" * 55)
        print("\n--- Level 0: Local QR on each block ---")

    Qs = {}          # store all Q factors
    R_current = []   # R factors produced at this level

    for i in range(num_procs):
        row_start = i * block_size
        row_end   = (i + 1) * block_size
        A_block   = A[row_start:row_end, :]          # sub-indexing: rows of proc i

        Q_i, R_i = qr(A_block)                      # local QR (numpy built-in)

        Qs[(0, i)] = Q_i
        R_current.append(R_i)                        # keep only R (n x n)

        if verbose:
            print(f"  Proc {i}: A[{row_start}:{row_end}, :] -> "
                  f"Q shape {Q_i.shape}, R shape {R_i.shape}")

    # ----------------------------------------------------------------
    # Reduction tree: log2(num_procs) levels
    # At each level, pairs of R matrices are stacked and QR'd
    # ----------------------------------------------------------------
    level     = 1
    active    = list(range(num_procs))  # active processor indices

    while len(R_current) > 1:
        if verbose:
            print(f"\n--- Level {level}: Reduction ({len(R_current)} -> "
                  f"{len(R_current)//2} R matrices) ---")

        R_next   = []
        new_active = []

        for j in range(0, len(R_current), 2):
            # Stack two adjacent R matrices: shape (2n x n)
            R_stacked = np.vstack([R_current[j], R_current[j + 1]])

            Q_j, R_j = qr(R_stacked)               # QR of stacked R's

            proc_id = active[j]                     # label by owning processor
            Qs[(level, proc_id)] = Q_j
            R_next.append(R_j)
            new_active.append(proc_id)

            if verbose:
                print(f"  Merge procs {active[j]},{active[j+1]}: "
                      f"stacked shape {R_stacked.shape} -> "
                      f"Q {Q_j.shape}, R {R_j.shape}")

        R_current = R_next
        active    = new_active
        level    += 1

    R_final = R_current[0]   # upper-triangular factor of A

    if verbose:
        print(f"\n==> Final R shape: {R_final.shape}")

    return R_final, Qs

---
## Part 2: Correctness Verification

We verify TSQR by checking two things:

1. **R matches reference**: compare the absolute values of R from TSQR vs `numpy.linalg.qr(A)` directly (signs may differ due to Householder convention)
2. **Residual** $\|A - Q_{\text{ref}} R_{\text{TSQR}}\|_F$: if R is correct, then $Q_{\text{ref}} = A R^{-1}$ should be orthogonal

In [ ]:
def verify_tsqr(A, R_tsqr, verbose=True):
    """
    Verify TSQR result against numpy's full QR as ground truth.

    Checks:
      (a) |R_tsqr| ≈ |R_ref|  (element-wise, sign-invariant)
      (b) ||A - Q_ref R_tsqr||_F / ||A||_F  (relative residual)
      (c) ||I - Q^T Q||_F  (orthogonality of Q_ref = A R_tsqr^{-1})
    """
    m, n = A.shape

    # Reference QR from numpy
    Q_ref, R_ref = qr(A, mode='reduced')   # Q: m x n, R: n x n

    # (a) R comparison (sign-invariant)
    R_diff = np.abs(R_tsqr) - np.abs(R_ref)
    r_err  = norm(R_diff, 'fro') / norm(R_ref, 'fro')

    # (b) Residual: reconstruct Q from A and R_tsqr via least squares
    #     Q* = A R^{-1}  (since A = QR => Q = AR^{-1})
    #     Use numpy solve for numerical stability
    Q_tsqr = np.linalg.solve(R_tsqr.T, A.T).T  # A @ inv(R)
    residual = norm(A - Q_tsqr @ R_tsqr, 'fro') / norm(A, 'fro')

    # (c) Orthogonality of recovered Q
    orth_err = norm(Q_tsqr.T @ Q_tsqr - np.eye(n), 'fro')

    if verbose:
        print("\n=== Verification ===")
        print(f"  Relative |R| difference vs numpy QR : {r_err:.2e}")
        print(f"  Relative residual ||A - QR||/||A||  : {residual:.2e}")
        print(f"  Orthogonality error ||I - Q^TQ||    : {orth_err:.2e}")
        tol = 1e-10
        ok  = (r_err < tol) and (residual < tol) and (orth_err < tol)
        print(f"  PASS: {ok}")

    return r_err, residual, orth_err


# --- Run a small concrete example ---
m, n = 1200, 6    # tall-skinny: 1200 rows, 6 cols, 4 processors of 300 rows each
A    = np.random.randn(m, n)

R_tsqr, Qs = tsqr(A, num_procs=4, verbose=True)
verify_tsqr(A, R_tsqr, verbose=True)

---
## Part 3: Step-by-Step Walkthrough on a Tiny Matrix

To make the linear algebra transparent, we trace through a $16 \times 2$ example by hand.

In [ ]:
print("=" * 55)
print("Tiny example: A is 16 x 2, split into 4 blocks of 4 rows")
print("=" * 55)

A_tiny = np.random.randn(16, 2)

# Level 0: local QR on each block
blocks = [A_tiny[i*4:(i+1)*4, :] for i in range(4)]
R0 = []
for i, blk in enumerate(blocks):
    Q_i, R_i = qr(blk)
    R0.append(R_i)
    print(f"\nProc {i}: A_block =\n{np.round(blk,3)}")
    print(f"  R_{i} =\n{np.round(R_i,4)}")

# Level 1: merge (0,1) and (2,3)
print("\n--- Level 1 merge ---")
stacked_01 = np.vstack([R0[0], R0[1]])
stacked_23 = np.vstack([R0[2], R0[3]])
Q01, R01 = qr(stacked_01)
Q23, R23 = qr(stacked_23)
print(f"Stack [R0;R1] -> R_01 =\n{np.round(R01,4)}")
print(f"Stack [R2;R3] -> R_23 =\n{np.round(R23,4)}")

# Level 2: final merge
print("\n--- Level 2 merge (final) ---")
stacked_final = np.vstack([R01, R23])
Q_final, R_final = qr(stacked_final)
print(f"Stack [R01;R23] -> R_final =\n{np.round(R_final,4)}")

# Compare with numpy reference
_, R_ref = qr(A_tiny, mode='reduced')
print(f"\nnumpy reference R =\n{np.round(R_ref,4)}")
print(f"\n|R_final| vs |R_ref| difference: {norm(np.abs(R_final)-np.abs(R_ref)):.2e}")